# Demo Notebook for Bayesian Hyperparameter Searches

In [ ]:
import datajoint as dj

dj.config["enable_python_native_blobs"] = True
dj.config['nnfabrik.schema_name'] = "nnfabrik_hypersearch_demo"
schema = dj.schema("nnfabrik_hypersearch_demo")

import nnfabrik
import torch
from nnfabrik.main import *
import os
from os import listdir
from os.path import isfile, join
import datajoint as dj

import os
import torch
import numpy as np
import pickle 

import nnfabrik
from nnfabrik import main, builder

import nnvision



In [ ]:
# import the Bayesian Class from nnfabrik
from nnfabrik.utility.hypersearch import Bayesian


In [ ]:
# For Dataset, Trainer, and Model, the parameters that should be optimized should be defined in the _auto dictionarys
# Here are example usages for all the entries.
# Those parameters should not be defined in the respective configs

model_config_auto = dict(

    gamma_readout={"type": "range", "bounds": [1e-6, 1e-1], "log_scale": True},
    init_noise={"type": "range", "bounds": [1e-6, 1e-1], "log_scale": True},   
)
dataset_config_auto = dict(
    batch_size={"type": "choice", "values": [50, 64]}
)

trainer_config_auto = dict(
    lr_init={"type": "range", "bounds": [0.001, 0.005], "log_scale": True},
              )

# Add Fabrikant

In [ ]:
# # change this entry to reflect your datajoint username
Fabrikant().insert1(dict(fabrikant_name='kwilleke',
                          email="konstantin.willeke@gmail.com",
                          affiliation='sinzlab',
                          dj_username="kwilleke"))
Seed().insert1(dict(seed=1000))

# Add the Dataset

In [ ]:

basepath = '/data/monkey/toliaslab/CSRF19_V1'
neuronal_data_path = os.path.join(basepath, 'neuronal_data/')
neuronal_data_files = [neuronal_data_path+f for f in listdir(neuronal_data_path) if isfile(join(neuronal_data_path, f))]
image_file = os.path.join(basepath, 'images/CSRF19_V1_images.pickle')
image_cache_path = os.path.join(basepath, 'images/individual')


dataset_fn = 'nnvision.datasets.monkey_static_loader'
dataset_config = dict(dataset='CSRF19_V1',
                               neuronal_data_files=neuronal_data_files[:15],
                               image_cache_path=image_cache_path,
                               crop=80,
                               subsample=1,
                               seed=1000,
                               time_bins_sum=12,
                               batch_size=128,)
dataset_config_auto = dict()

# Add the Model

In [ ]:
model_fn = 'nnvision.models.se_core_spatialXfeature_readout'
model_config =  {'pad_input': False,
   'stack': -1,
   'depth_separable': True,
   'input_kern': 24,
   'gamma_input': 20,
   'hidden_dilation': 1,
   'hidden_kern': 7,
   'hidden_channels': 32}

model_config_auto = dict(

    gamma_readout={"type": "range", "bounds": [1e-6, 1e-1], "log_scale": True},
    init_noise={"type": "range", "bounds": [1e-6, 1e-1], "log_scale": True},   
)

# Add the Trainer

In [ ]:
trainer_fn = 'nnvision.training.nnvision_trainer'
trainer_config = dict(max_iter=100,
                      verbose=False, 
                      lr_decay_steps=4,
                      avg_loss=False, 
                      patience=5,
                      lr_init=.0045)
trainer_config_auto = dict(
              )

# Run Bayesian Search

In [ ]:
# Starting the bayesian Search:
# The number of total trials can be varied, but 200 trials is usually sufficient

In [ ]:
autobayes = Bayesian(dataset_fn, dataset_config, dataset_config_auto,
              model_fn, model_config, model_config_auto,
              trainer_fn, trainer_config, trainer_config_auto, architect="kwilleke", trained_model_table='nnfabrik.my_trained_model.TrainedModel', total_trials=200)

best_parameters, _, _, _ = autobayes.run()